## 과제 4: 직접 빌드한 컨테이너로 데이터 처리 수행

이전 노트북에서는 Amazon SageMaker Processing 및 scikit-learn 기본 제공 컨테이너를 데이터 처리에 사용했습니다.

이 노트북에서는 직접 빌드한 처리 컨테이너로 scikit-learn 스크립트를 실행하는 데 필요한 환경을 설정합니다.

구체적으로는 Docker 이미지를 직접 생성하고 처리 컨테이너를 빌드한 후 Amazon SageMaker Python SDK의 **ScriptProcessor** 클래스를 사용하여 컨테이너 내에서 scikit-learn 전처리 스크립트를 실행합니다.

그리고 마지막으로 Amazon Simple Storage Service(Amazon S3)에 저장된 데이터 처리 결과를 검증합니다.

### 과제 4.1: 환경 설정 

이 과제에서는 필요한 라이브러리와 종속성을 설치합니다. 

처리 작업의 출력을 저장할 Amazon S3 버킷을 설정하고 SageMaker Processing 작업을 실행할 실행 역할도 가져옵니다.

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

### 과제 4.2: 처리 컨테이너 생성

이 과제에서는 Dockerfile을 사용하여 scikit-learn 컨테이너를 정의하고 생성합니다.

### 과제 4.3: Dockerfile 생성 

이 과제에서는 Docker 디렉터리를 생성한 후 처리 컨테이너를 생성하는 데 사용되는 Dockerfile을 추가합니다. scikit-learn 컨테이너를 생성할 것이므로 pandas 및 scikit-learn을 설치합니다.

In [ ]:
%mkdir docker

In [ ]:
%%writefile docker/Dockerfile
FROM public.ecr.aws/docker/library/python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

### 과제 4.4: 컨테이너 이미지 빌드

이 과제에서는 Amazon SageMaker Studio 이미지 빌드 CLI(Command Line Interface)를 사용하여 사용자 지정 컨테이너 이미지를 생성합니다. 

Amazon SageMaker Studio 이미지 빌드 CLI를 사용하면 Amazon SageMaker 호환 Docker 이미지를 SageMaker Studio 환경에서 직접 빌드할 수 있습니다.

SageMaker Studio 이미지 빌드 패키지를 설치합니다.

In [ ]:
%pip install sagemaker-studio-image-build

Dockerfile이 포함된 디렉터리로 이동하여 sm-docker build 명령을 실행합니다. 이 명령은 빌드 출력을 자동으로 로깅하고 Docker 이미지의 **이미지 URI**를 반환합니다. 이 명령 실행을 완료하려면 약 2분이 소요됩니다.

In [ ]:
%%sh

cd docker

sm-docker build .

다음으로 **이미지 URI**를 복사하여 원하는 텍스트 편집기에 붙여 넣습니다. 
이 **이미지 URI**를 사용하여 **ScriptProcessor** 클래스를 생성합니다.

### 과제 4.5: SageMaker 처리 작업 실행

이 과제에서는 이전 노트북에서 사용했던 것과 같은 전처리된 데이터 집합을 사용합니다.

In [ ]:
#import-data
shape=pd.read_csv("data/adult_data.csv", header=None)
shape.sample(5)



그런 다음 SageMaker ScriptProcessor 클래스를 사용하여 처리 스크립트를 처리 작업으로 정의하고 실행합니다. 이 클래스에 관한 자세한 내용은 [SageMaker ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor)를 참조하세요.

ScriptProcessor 클래스를 생성하려면 다음 파라미터를 구성합니다.
- **base_job_name**: 처리 작업 이름의 접두사
- **command**: 실행할 명령(명령줄 플래그와 함께 구성)
- **image_uri**: 처리 작업에 사용할 Docker 이미지의 URI
- **role**: SageMaker 실행 역할
- **instance_count**: 처리 작업을 실행할 인스턴스의 수
- **instance_type**: 처리 작업에 사용되는 Amazon Elastic Compute Cloud(Amazon EC2) 인스턴스의 유형

텍스트 편집기에서 다음 코드의 **REPLACE_IMAGE_URI**를 실제 URI로 바꿉니다.

In [ ]:
#sagemaker-script-processor
from sagemaker.processing import ScriptProcessor

# create a ScriptProcessor
script_processor = ScriptProcessor(
    base_job_name="own-processing-container",
    command=["python3"],
    image_uri="REPLACE_IMAGE_URI",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)


다음으로는 ScriptProcessor.run() 메서드를 사용하여 **sklearn_preprocessing.py** 스크립트를 처리 작업으로 실행합니다. 이 스크립트는 과제 3에서 사용한 것과 같은 스크립트이지만 이번에는 기본 이미지에서 빌드한 사용자 지정 컨테이너에서 해당 스크립트를 실행합니다. 이 메서드에 관한 자세한 내용은 [ScriptProcessor.run()](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor.run)을 참조하세요.

처리 작업을 실행하기 위해 다음 파라미터를 구성합니다.
- **code**: 전처리 스크립트의 경로 
- **inputs**: 전처리 스크립트용 입력 데이터의 경로(Amazon S3 입력 위치)
- **outputs**: 전처리 스크립트의 출력 경로(Amazon S3 출력 위치)
- **arguments**: 전처리 스크립트의 명령줄 인수(예: 훈련 테스트 분할 비율)

처리 작업을 완료하려면 약 4-5분이 소요됩니다.

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
script_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "adult_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                         source="/opt/ml/processing/test",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)

### 과제 4.6: 데이터 처리 결과 검증

실행한 처리 작업의 출력을 검증합니다. 이렇게 하려면 훈련 및 테스트 출력 데이터 집합의 첫 5개 행을 확인합니다.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

### 마무리

축하합니다! 처리 컨테이너를 직접 빌드했으며 SageMaker Processing을 사용하여 처리 작업을 실행했습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 지침으로 돌아갑니다.